In [1]:
from casadi import *
import matplotlib.pyplot as plt
import numpy as np

## Pendulum exercise
$$
\begin{split}\begin{array}{lc}
\begin{array}{l}
\text{minimize:} \\
x(\cdot) \in \mathbb{R}^2, \, u(\cdot) \in \mathbb{R}
\end{array}
\quad \displaystyle \int_{t=0}^{T}{cos(x_0) \, dt}
\\
\\
\text{subject to:} \\
\\
\begin{array}{ll}
\left\{
\begin{array}{l}
\dot{x}_0 = x_1 \\
\dot{x}_1 = u - sin(x_0) \\
-u_{max} \le u \le u_{max} , \quad
\end{array} \right. & \text{for} \, 0 \le t \le T \\
x_0(0)=0, \quad x_1(0)=0, x_0(T) = pi/2 , x_1(T) = 0
\end{array}
\end{array}\end{split}
$$
with $T=10$.

siendo $$x_0 = \theta$$ $$x_1 = \theta'$$

In [2]:
x = MX.sym('x', 2)
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

In [3]:
rhs = vertcat(x[1], u-sin(x[0]))

In [4]:
F = Function('F', [x, u], [rhs])

In [24]:
k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [25]:
#new_x_expr = x + dt * F(x, u)

In [26]:
new_x = Function('New_x', [x, u, dt], [new_x_expr])

In [27]:
N = 100
T = 10

In [28]:
opti = Opti()

In [29]:
X = opti.variable(N+1,2)

In [30]:
U = opti.variable(N)

In [31]:
u_m = opti.parameter()

In [32]:
X.shape

(101, 2)

In [33]:
cost = sum1(cos(X[:,0]))

In [34]:
cost

MX(mac(ones(1x101),cos(opti1_x_1[:101]),0))

In [35]:
opti.minimize(cost)

In [36]:
opti.solver('ipopt')

In [37]:
opti.set_value(u_m, 20)

In [38]:
opti.subject_to(X[0,:].T == [0, 0])

In [39]:
opti.subject_to(X[-1,:].T == [pi, 0])
#opti.subject_to(X[-1,0].T == [pi])

In [40]:
for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii], U[ii], T/N))
    #opti.subject_to(U[ii] < u_m)
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [41]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))

In [42]:
opti.set_initial(X[:,1], pi/N)

In [43]:
sol = opti.solve()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      604
Number of nonzeros in inequality constraint Jacobian.:      100
Number of nonzeros in Lagrangian Hessian.............:      301

Total number of variables............................:      302
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      204
Total number of inequality constraints...............:      100
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      100
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'

In [ ]:
sol.value

In [ ]:
opti.debug.show_infeasibilities()